In [1]:
from ioMicro import *

In [8]:
save_folder =r'\\192.168.0.10\bbfishdc13\DCBBL1_3_2_2023\MERFISH_Analysis'
fl_dapis = []#glob.glob(r'\\192.168.0.10\bbfishdc13\DCBBL1_3_10_2023__GFP\H9_MER_set1\Conv_zscan__*.zarr')
for i in range(4):
    fl_dapis += glob.glob(r'Z:\DCBBL1_3_2_2023\MERFISH\H9_MER_set1\Conv_zscan__*.zarr'.replace('_set1','_set'+str(i+1)))


In [13]:
fl_dapi = fl_dapis[30]
final_segmentation(fl_dapi,
                        analysis_folder=save_folder,
                        plt_val=True,
                        rescz = 5,trimz=0, resc=4,p99=2000,force=True)

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 14.54it/s]


Saved file:\\192.168.0.10\bbfishdc13\DCBBL1_3_2_2023\MERFISH_Analysis\Segmentation\Conv_zscan__030--H9_MER_set1--dapi_segm__segim.png


In [14]:
!"\\192.168.0.10\bbfishdc13\DCBBL1_3_2_2023\MERFISH_Analysis\Segmentation\Conv_zscan__030--H9_MER_set1--dapi_segm__segim.png"

In [ ]:
for fl_dapi in fl_dapis:
    try:
        final_segmentation(fl_dapi,
                            analysis_folder=save_folder,
                            plt_val=False,
                            rescz = 5,trimz=0, resc=4,p99=2000,force=False)
    except:
        print("Failed")

  0%|                                                                                            | 0/8 [00:05<?, ?it/s]


Failed


 38%|███████████████████████████████▌                                                    | 3/8 [00:29<00:52, 10.44s/it]

In [6]:
!"\\192.168.0.10\bbfishdc13\DCBBL1_3_10_2023__GFP\MERFISH_Analysis\Segmentation\Conv_zscan__177--H9_MER_set2--dapi_segm__segim.png"

In [4]:
!"\\192.168.0.10\bbfishdc13\DCBBL1_3_10_2023__GFP\MERFISH_Analysis\Segmentation\Conv_zscan__012--P1__Igfbp_Aldh1l1_Ptbp1__set1--dapi_segm__segim.png"

In [ ]:
fl = r'\\192.168.0.10\bbfishdc13\DCBBL1_3_10_2023__GFP\H9_MER_set1\Conv_zscan__012.zarr'
im_dapi = read_im(fl)[-1]#[::6]

In [80]:
fov = os.path.basename(fl)
set_ = '_'+os.path.basename(os.path.dirname(fl)).split('_')[-1]

drift_fl = save_fld+os.sep+'drift_'+fov.split('.')[0]+'--'+set_+'.pkl'
drifts,all_flds,fov = pickle.load(open(drift_fl,'rb'))
tag = os.path.basename(os.path.dirname(fl))
tags = [os.path.basename(fld) for fld in all_flds]
drift = drifts[tags.index(tag)][0]

save_fld = r'\\192.168.0.10\bbfishdc13\DCBBL1_3_10_2023__GFP\MERFISH_Analysis'
dec = decoder_simple(save_fld,fov=fov, set_=set_)
dec.load_decoded()
XH = dec.XH_pruned.copy()
XHM = np.mean(XH,axis=1)
X = XHM[:,:3].astype(int)-drift
shape = im_dapi.shape
im_ = np.zeros(shape)
keep = np.all((X>=0)&(X<shape),axis=-1)
#gn = 'Bsn'
#ign = np.where(dec.gns_names==gn)[0][0]
#icds_ = icds[np.argsort(ncts)[::-1]][:10]
#icds_ = [ign]
#keep&= np.in1d(dec.icodesN,icds_)
im_[tuple(X[keep].T)]=XHM[keep,-3]
from scipy.ndimage import gaussian_filter
im__ = gaussian_filter(im_[:,:,:], 4)

Loaded decoded: -1.7433724403381348


In [51]:
V = napari.view_image(im_dapi)
V.add_image(im__)

<Image layer 'im__' at 0x262b0d417f0>

In [168]:
im_dapi_ = im_dapi[len(im_dapi)//2]
im_ = np.array(im_dapi_,dtype=np.float32)
img = (cv2.blur(im_,(3,3))-cv2.blur(im_,(150,150)))[::resc,::resc]
p1 = np.percentile(img,1)
#p1 = np.percentile(img,1)
#p1=0
if p99 is None:
    p99 = np.percentile(img,99.9)
p99=2000
img_dapi = np.array(np.clip((img-p1)/(p99-p1),0,1),dtype=np.float32)
im_sig = im__[len(im_dapi)//2][::resc,::resc].copy()
im_sig = np.clip(im_sig*50/10000,0,1)
imf = np.dstack([im_sig,im_sig,img_dapi])

In [241]:
im_dapi_ = im_dapi[len(im_dapi)//2]#-(im_dapi[len(im_dapi)//2-1]+im_dapi[len(im_dapi)//2+1])/2
im_ = np.array(im_dapi_,dtype=np.float32)
img = (cv2.blur(im_,(3,3))-cv2.blur(im_,(150,150)))[::resc,::resc]
p1 = np.percentile(img,1)
#p1 = np.percentile(img,1)
#p1=0
if p99 is None:
    p99 = np.percentile(img,99.9)
p99=2000
img_dapi = np.array(np.clip((img-p1)/(p99-p1),0,1),dtype=np.float32)
#im_sig = im__[len(im_dapi)//2][::resc,::resc].copy()
#im_sig = np.clip(im_sig*50/10000,0,1)
#imf = np.dstack([im_sig,im_sig,img_dapi])

In [242]:
napari.view_image(img_dapi)

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 374.5, 374.5), zoom=1.625133333333333, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(1.0, 1.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=2, ndisplay=2, last_used=0, range=((0.0, 750.0, 1.0), (0.0, 750.0, 1.0)), current_step=(375, 375), order=(0, 1), axis_labels=('0', '1')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'img_dapi' at 0x2650529c280>], scale_bar=ScaleBar(visible=False, colored=False, color=array([1., 0., 1., 1.], dtype=float32), ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10.0, box=False, box_color=array([0. , 0. , 0. , 0.6], dtype=float32), unit=None), text_overlay=TextOverlay(visible=False, color=array([0.5, 0.5, 0.5, 1. ], dtype=float32), font_size=10.0, position=<TextOverlayPosition.TOP_LEFT: 'top_left'>, text=''), overl

In [204]:
from cellpose import models, io,utils
imf = np.dstack([img_dapi,im_sig,img_dapi])
resc=4
sz_min_2d=400
sz_cell=22
use_gpu=True
model='cyto'
#p99=None
model = models.Cellpose(gpu=use_gpu, model_type=model)
masks, flows, styles, diams = model.eval(img_dapi, diameter=sz_cell, channels=[0,0],
                                         flow_threshold=20,cellprob_threshold=-30,min_size=100,normalize=False)

In [233]:
from scipy import ndimage
umsks = np.unique(masks)
means = ndimage.mean(im_[::resc,::resc],masks,index=umsks)
th = (np.percentile(means[1:],25)+means[0])/2
bad = np.in1d(masks,umsks[means<th])
masks_ = masks.copy()
masks_[bad.reshape(masks_.shape)]=0

In [234]:
import napari
viewer = napari.Viewer()
viewer.add_image(im_[::resc,::resc])
#viewer.add_image(im_sig)
viewer.add_labels(masks)
viewer.add_labels(masks_)

<Labels layer 'masks_' at 0x264fbbcd070>

In [67]:
all_flds[np.argmin([np.sum(np.abs(drft[0])) for drft in drifts])]

'\\\\192.168.0.10\\bbfishdc13\\DCBBL1_3_10_2023__GFP\\H16_MER_set2'

In [183]:
flows[0].shape

(750, 750, 3)

In [40]:
from cellpose import models, io,utils
resc=4
sz_min_2d=400
sz_cell=20
use_gpu=True
model='cyto2'
p99=None

model = models.Cellpose(gpu=use_gpu, model_type=model)
#decided that resampling to the 4-2-2 will make it faster
#im_dapi_3d = im_dapi[::rescz,::resc,::resc].astype(np.float32)
chan = [0,0]
masks_all = []
flows_all = []
from tqdm import tqdm
for im in tqdm(im_dapi[[0]]):
    im_ = np.array(im,dtype=np.float32)
    img = (cv2.blur(im_,(2,2))-cv2.blur(im_,(250,250)))[::resc,::resc]
    p1 = np.percentile(img,1)
    if p99 is None:
        p99 = np.percentile(img,99.9)
    img = np.array(np.clip((img-p1)/(p99-p1),0,1),dtype=np.float32)
    masks, flows, styles, diams = model.eval(img, diameter=sz_cell, channels=chan,
                                         flow_threshold=0.85,cellprob_threshold=-2,min_size=100,normalize=False)


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:08<00:00,  8.18s/it]


In [41]:
import napari
viewer = napari.Viewer()
viewer.add_image(img)
viewer.add_labels(masks)

<Labels layer 'masks' at 0x262b0c129d0>

In [ ]:
def final_segmentation(fl_dapi,
                        analysis_folder=r'X:\DCBB_human__11_18_2022_Analysis',
                        plt_val=True,
                        rescz = 5,trimz=2, resc=4,p99=4500):
    segm_folder = analysis_folder+os.sep+'Segmentation'
    if not os.path.exists(segm_folder): os.makedirs(segm_folder)

    save_fl  = segm_folder+os.sep+os.path.basename(fl_dapi).split('.')[0]+'--'+os.path.basename(os.path.dirname(fl_dapi))+'--dapi_segm.npz'

    if not os.path.exists(save_fl):
        im = read_im(fl_dapi)
        #im_mid_dapi = np.array(im[-1][im.shape[1]//2],dtype=np.float32)
        im_dapi = im[-1,::rescz][trimz:-trimz]

        im_seg_2 = standard_segmentation(im_dapi,resc=resc,sz_min_2d=100,sz_cell=20,use_gpu=True,model='cyto2',p99=p99)
        shape = np.array(im[-1].shape)
        np.savez_compressed(save_fl,segm = im_seg_2,shape = shape)

In [ ]:
standard_segmentation(im_dapi,resc=4,sz_min_2d=400,sz_cell=25,use_gpu=True,model='cyto2',p99=None)

In [4]:
??standard_segmentation

In [44]:
save_folder =r'\\192.168.0.10\bbfishdc13\DCBBL1_3_10_2023__GFP\MERFISH_Analysis'
fl_dapi = r'\\192.168.0.10\bbfishdc13\DCBBL1_3_10_2023__GFP\P1__Igfbp_Aldh1l1_Ptbp1__set1\Conv_zscan__012.zarr'
final_segmentation(fl_dapi,
                    analysis_folder=save_folder,
                    plt_val=True,
                    rescz = 5,trimz=2, resc=4,p99=4500)

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 11.94it/s]


Saved file:\\192.168.0.10\bbfishdc13\DCBBL1_3_10_2023__GFP\MERFISH_Analysis\Segmentation\Conv_zscan__012--P1__Igfbp_Aldh1l1_Ptbp1__set1--dapi_segm__segim.png
